In [1]:
import torch
import numpy as np

In [2]:
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Input
x = torch.tensor([3.0])

In [3]:
# Forward pass
y_pred = w *x * b
print(f"y_pred: {y_pred.item()}")

y_pred: 6.0


In [4]:
# Loss
y_true = torch.tensor([10.0])
loss = (y_pred - y_true) ** 2
print(f"Loss: {loss.item()}")

Loss: 16.0


In [5]:
print(f"Avant backward: w.grad = {w.grad}")

Avant backward: w.grad = None


In [6]:
loss.backward()

In [7]:
print(f"APrès backward: w.grad = {w.grad}") # Gradient calculé
print(f"Après backward: b.grad = {b.grad}")

APrès backward: w.grad = tensor([-24.])
Après backward: b.grad = tensor([-48.])


In [8]:
# Update manuel des poids
learning_rate = 0.01
with torch.no_grad():
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

In [9]:
print(f"Nouveaux poids: w = {w}, b = {b}")

Nouveaux poids: w = tensor([2.2400], requires_grad=True), b = tensor([1.4800], requires_grad=True)


In [10]:
# Réinitialiser les gradients (IMPORTANT):
w.grad.zero_()
b.grad.zero_()
print(f"Après zero_: w.grad = {w.grad}, b.grad = {b.grad}")

Après zero_: w.grad = tensor([0.]), b.grad = tensor([0.])


In [11]:
import torch.nn as nn
perceptron = nn.Linear(1, 1)

# Voir les poids initiaux
print(f"Poids: {perceptron.weight}")
print(f"Biais: {perceptron.bias}")

Poids: Parameter containing:
tensor([[0.8234]], requires_grad=True)
Biais: Parameter containing:
tensor([0.4510], requires_grad=True)


In [12]:
# Forward
x = torch.tensor([[3.0]]) #Shape (batch_size, features)
print(f"input: {x}")
y_pred = perceptron(x)
print(y_pred)

input: tensor([[3.]])
tensor([[2.9212]], grad_fn=<AddmmBackward0>)


In [13]:
# Loss
y_true = torch.tensor([[10.0]])
criterion = nn.MSELoss()
loss = criterion(y_pred, y_true)

In [14]:
# Backward
loss.backward()

In [15]:
print(f"Gradient du poids: {perceptron.weight.grad}")
print(f"Gradient du biais: {perceptron.bias.grad}")

Gradient du poids: tensor([[-42.4726]])
Gradient du biais: tensor([-14.1575])


In [16]:
import torch.optim as optim
optimizer = optim.SGD(perceptron.parameters(), lr=0.01)

In [17]:
optimizer.step()

In [18]:
print(f"Nouveaux poids: {perceptron.weight}")
print(f"Nouveaux biais: {perceptron.bias}")

Nouveaux poids: Parameter containing:
tensor([[1.2481]], requires_grad=True)
Nouveaux biais: Parameter containing:
tensor([0.5925], requires_grad=True)


In [19]:
optimizer.zero_grad()

In [20]:
print(f"Gradient du poids: {perceptron.weight.grad}")
print(f"Gradient du biais: {perceptron.bias.grad}")

Gradient du poids: None
Gradient du biais: None


In [21]:
# For sur 1 neurone (se rapproche de plus en plus à la valeur attendu)

In [22]:
# Création des modèles

In [23]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)#connecté à la neurone précédente donc prend le nombre de neurones sorties de la precedente en entrée
        self.fc3 = nn.Linear(3, 1)#pareil
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [24]:
model = SimpleNet()

In [25]:
model#tout ce qu'il contient dans le constructeur

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [26]:
x = torch.tensor([1.0, 2.0])
y = torch.tensor([5.0])
y_pred = model(x)
loss = nn.MSELoss()(y_pred,y)

loss.backward()

In [27]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import matplotlib.pyplot as plt

In [28]:
training_data = datasets.MNIST(
    root = "data",#ou les stocker -< va le creer lui meme
    train = True,#dataset d'entrainement ou de test
    download = True,#est ce que le dataset a été téléchargé? Non alors le telecharger download = True
    transform = ToTensor()
)

test_data = datasets.MNIST(
    root = "data",#ou les stocker -< va le creer lui meme
    train = False,#dataset d'entrainement ou de test
    download = False,#est ce que le dataset a été téléchargé? Non alors le telecharger download = True
    transform = ToTensor()
)

In [29]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [30]:
print(training_data.classes)
print(training_data.data.size())#6000 images de 28x28

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [31]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()# pour que les matrices soient ajusté entre elles (regle de largeur longuer quand on fait calcul A larg = B long)
        self.fc = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits

In [32]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [33]:
X = torch.rand(1, 28, 28, device = device)# mettre device sinon erreur "cpu is expected but no device has benn given"
logits = model(X)

In [34]:
logits

tensor([[ 0.1483, -0.0391, -0.0262,  0.0318,  0.0865, -0.0290,  0.0541,  0.1666,
          0.0081, -0.1647]], grad_fn=<AddmmBackward0>)

In [35]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.1128, 0.0935, 0.0947, 0.1004, 0.1060, 0.0945, 0.1027, 0.1149, 0.0980,
         0.0825]], grad_fn=<SoftmaxBackward0>)

In [36]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([7])

In [37]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [38]:
tuple_test = (1, 2, "a")
a, b, c = tuple_test
#batch_idx, batch -> (X,y)

In [39]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5}/{size:>5}]")

In [40]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    print("Done!")

Epoch 1
---------------------------------------


c:\Users\Talyha Moreau\Documents\IIM\IA\Projet_IA\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss: 2.293447 [   64/60000]
loss: 2.308760 [ 6464/60000]
loss: 2.289027 [12864/60000]
loss: 2.296826 [19264/60000]
loss: 2.287796 [25664/60000]
loss: 2.274779 [32064/60000]
loss: 2.283822 [38464/60000]
loss: 2.262749 [44864/60000]
loss: 2.267436 [51264/60000]
loss: 2.279583 [57664/60000]
Test Error: 
 Accuracy 28.6%, Avg loss: 2.258159 

Done!
Epoch 2
---------------------------------------
loss: 2.251084 [   64/60000]
loss: 2.252446 [ 6464/60000]
loss: 2.229422 [12864/60000]
loss: 2.242142 [19264/60000]
loss: 2.224797 [25664/60000]
loss: 2.243906 [32064/60000]
loss: 2.242865 [38464/60000]
loss: 2.223522 [44864/60000]
loss: 2.202617 [51264/60000]
loss: 2.206362 [57664/60000]
Test Error: 
 Accuracy 46.2%, Avg loss: 2.197294 

Done!
Epoch 3
---------------------------------------
loss: 2.205893 [   64/60000]
loss: 2.182894 [ 6464/60000]
loss: 2.182884 [12864/60000]
loss: 2.164634 [19264/60000]
loss: 2.137843 [25664/60000]
loss: 2.148076 [32064/60000]
loss: 2.160712 [38464/60000]
loss: 2

In [42]:
import torch.nn.functional as F

class ImageClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [43]:
torch_model = ImageClassifierModel()
# Create example inputs for exporting the model. The inputs should be a tuple of tensors.
example_inputs = (torch.randn(1, 1, 32, 32),)
onnx_program = torch.onnx.export(torch_model, example_inputs, dynamo=True)
onnx_program.save("image_classifier_model.onnx")

[torch.onnx] Obtain model graph for `ImageClassifierModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ImageClassifierModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
